In [5]:
from serpapi import GoogleSearch

def google_image_api(query, ijn, num_images=10000):
    params = {
        "q": query,
        "tbm": "isch",
        "num": num_images,
        "api_key": "cea99d066409fd7850a0d5c37c1ebf1c5d6dfa331d909fffa52a85047315b346",
        "ijn":ijn
    }
    
    search = GoogleSearch(params)
    results = search.get_dict()
    image_urls = [item["original"] for item in results["images_results"][:num_images]]

    return image_urls



In [8]:
image_links[0]

'https://i.etsystatic.com/37043708/r/il/c2b5e7/4116940488/il_fullxfull.4116940488_koxk.jpg'

In [6]:
image_links_1 = google_image_api("car with baby sticker",1, 100)

In [10]:
image_links_1[0]

'https://www.pennyandprince.com/uploads/5/1/3/3/51331257/s474748300754869697_p23_i3_w2550.jpeg'

In [22]:
image_list = []
for i in range(0,99):
    image_links_k = google_image_api("car with baby sticker",i, 100)
    image_list.append(image_links_k)

KeyError: 'images_results'

In [28]:
image_list_all = []
for i in image_list:
    image_list_all += i

In [30]:
import os
import json
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

# 设置下载路径
OUTPUT_DIR = "downloaded_images"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 下载单张图片
def download_image(url, index):
    try:
        response = requests.get(url, stream=True, timeout=10)
        if response.status_code == 200:
            ext = url.split(".")[-1]  # 获取图片扩展名
            if ext.lower() not in ["jpg", "jpeg", "png"]:
                ext = "jpg"  # 默认 jpg 格式
            
            image_path = os.path.join(OUTPUT_DIR, f"image_{index}.{ext}")
            with open(image_path, "wb") as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            return f"✅ {index}: 下载成功 -> {image_path}"
        else:
            return f"❌ {index}: 请求失败 {response.status_code}"
    except Exception as e:
        return f"❌ {index}: 下载失败 - {e}"

# 多线程批量下载
def bulk_download_images(image_links, max_threads=10):
    print(f"📥 开始下载 {len(image_links)} 张图片...")
    with ThreadPoolExecutor(max_threads) as executor:
        results = list(tqdm(executor.map(download_image, image_links, range(len(image_links))), total=len(image_links)))

    # 输出失败情况
    failed_downloads = [res for res in results if res.startswith("❌")]
    if failed_downloads:
        print("\n⚠ 失败列表：")
        for fail in failed_downloads[:10]:  # 仅显示前 10 个失败
            print(fail)
    print("🎉 所有可下载的图片已完成！")

bulk_download_images(image_list_all, max_threads=20)


📥 开始下载 823 张图片...


100%|██████████| 823/823 [00:23<00:00, 35.29it/s]


⚠ 失败列表：
❌ 3: 下载失败 - HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=10)
❌ 11: 请求失败 403
❌ 43: 请求失败 403
❌ 49: 请求失败 403
❌ 61: 请求失败 403
❌ 68: 请求失败 403
❌ 86: 请求失败 403
❌ 105: 请求失败 403
❌ 118: 请求失败 403
❌ 133: 下载失败 - HTTPSConnectionPool(host='www.gobizkorea.com', port=443): Max retries exceeded with url: /image/goodsImage.do?goods_no=GS2023092233250&image_se_code=MAIN_THUMB10A (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1002)')))
🎉 所有可下载的图片已完成！
